In [138]:
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import pandas as pd

import numpy as np 

import create_model
import pickle
import time 
import copy
import random
import data_formatting


def validate(train):
    
    if train == True:
        
        model = train_model
        mode = 'TRAIN'
        
    else:
        mode = 'DEV'
        model = dev_model

    encoder, decoder, predicted = session.run([model.encoder_inputs, model.decoder_targets, model.decoder_pred_train])            

    print ('Current mode:%s' % mode)
    for i, (e_in, dt_targ, dt_pred) in enumerate(zip( encoder, decoder, predicted)):

        print('  sample {}:'.format(i + 1))
        #print('    enc input           > {}'.format(e_in))
        print('    enc input           > {}'.format(data_formatting.decodeSent(e_in, inv_map)))

        #print('    dec input           > {}'.format(dt_targ))
        print('    dec input           > {}'.format(data_formatting.decodeSent(dt_targ, inv_map)))

        #print('    dec train predicted > {}'.format(dt_pred))
        print('    dec train predicted > {}'.format(data_formatting.decodeSent(dt_pred, inv_map)))

        if i >= 0: break

In [139]:
#dataset = 'enron'

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_enron_py35_seq_length_3_49_sample_4256_limited_vocab.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_enron_py35_seq_length_3_49_sample_4256_limited_vocab.pkl')

dataset = 'twitter'

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_25_sample_1764604_questions.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_25_sample_1764604_questions.pkl')

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_4_15_sample_134241_full.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_4_15_sample_134241_full.pkl')

vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_19_sample_21946_lem.pkl', 'rb'))
df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_19_sample_21946_lem.pkl')

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_25_sample_1901567_full.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_25_sample_1901567_full.pkl')

In [3]:
inv_map = data_formatting.createInvMap(vocab_dict)

In [4]:
def probSeq(seq, pos, n_gram_list, freq_counts):
    seq_len  = len(seq)

    if pos==seq_len: 

        return freq_counts
    
    else:

        token = seq[pos]
        n_gram_list = [n_gram for n_gram in n_gram_list if n_gram[pos]==token]
        freq_counts.append(len(n_gram_list))
        
        return probSeq(seq, pos+1, n_gram_list, freq_counts)    

In [5]:
df_all['alpha_Pair_0_encoding'] = df_all['alpha_Pair_0_tokens'].apply(lambda x: data_formatting.encodeSent(x, vocab_dict))
df_all['alpha_Pair_1_encoding'] = df_all['alpha_Pair_1_tokens'].apply(lambda x: data_formatting.encodeSent(x, vocab_dict))
df_all['Index'] = df_all.index.values

In [6]:
df_all_train = df_all.sample(frac=0.97, random_state=1)

df_all_dev = df_all[df_all['Index'].isin(df_all_train['Index'].values) == False]

df_all_test = df_all_dev.sample(frac=0.10, random_state=1)

df_all_dev = df_all_dev[df_all_dev['Index'].isin(df_all_test['Index'].values) == False]

In [7]:
print (df_all.shape[0], df_all_train.shape[0],  df_all_dev.shape[0], df_all_test.shape[0], len(vocab_dict))

17690 17159 478 53 17322


In [8]:
train_data = data_formatting.prepare_train_batch(df_all_train['alpha_Pair_0_encoding'].values, 
                                                    df_all_train['alpha_Pair_1_encoding'].values)

dev_data = data_formatting.prepare_train_batch(df_all_dev['alpha_Pair_0_encoding'].values, 
                                                    df_all_dev['alpha_Pair_1_encoding'].values)

test_data = data_formatting.prepare_train_batch(df_all_test['alpha_Pair_0_encoding'].values, 
                                                    df_all_test['alpha_Pair_1_encoding'].values)

In [9]:
train_model_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1, 'minibatch_size':64, 'n_threads':128,
          'beam_width':10, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1,
         }

In [10]:
dev_model_params = train_model_params
dev_model_params['encoder_input_keep'] = 1
dev_model_params['encoder_output_keep'] = 1
dev_model_params['decoder_input_keep'] = 1
dev_model_params['decoder_output_keep'] = 1

In [11]:
training_params = { 'vocab_lower':3, 'vocab_upper':train_model_params['vocab_size']-1, 
                    'n_epochs':500000}

In [12]:
#enc input           > ['i', 'll', 'be', 'there', 'tomorrow', '<EOS>']
#dec input           > ['last', 'game', 'of', 'the', 'regular', 'season', '<EOS>']

In [13]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import nn_ops
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import tensor_shape
from tensorflow.python.util import nest
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import tensor_array_ops

In [1259]:
tf.reset_default_graph()

inf_model_1_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1, 'minibatch_size':64, 'n_threads':128,
          'beam_width':1, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1,
         }

inf_model_2_params = {'n_cells':2, 'num_layers':1, 'embedding_size':5, 
            'vocab_size':7,
          'minibatch_size':64, 'n_threads':128,
          'beam_width':3, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1,
         }


inf_model_2_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1,
          'minibatch_size':64, 'n_threads':128,
          'beam_width':3, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1, 'anti_lm_weight':0
         }

#with tf.variable_scope('training_model'):

#    inf_model_1 = create_model.Model(inf_model_1_params, 'infer', None)
    
#with tf.variable_scope('training_model'):

#    inf_model_2 = create_model.Model(inf_model_2_params, 'infer', None)   

with tf.variable_scope('training_model'):

    inf_train = create_model.Model(inf_model_2_params, 'debug', None)   

In [1260]:
#names_to_vars_inf = [[v.op.name, v] for v in tf.all_variables()]

In [1261]:
names_to_vars = [[v.op.name, v] for v in tf.all_variables()]

Instructions for updating:
Please use tf.global_variables instead.


Instructions for updating:
Please use tf.global_variables instead.


In [1262]:
#[i[0] for i in names_to_vars_inf if 'forget' in i[0]]

In [1263]:
[i[0] for i in names_to_vars if 'forget' in i[0]]

['training_model/rnn/multi_rnn_cell/cell_0/layer_norm_basic_lstm_cell/forget/gamma',
 'training_model/rnn/multi_rnn_cell/cell_0/layer_norm_basic_lstm_cell/forget/beta',
 'training_model/rnn/multi_rnn_cell/cell_1/layer_norm_basic_lstm_cell/forget/gamma',
 'training_model/rnn/multi_rnn_cell/cell_1/layer_norm_basic_lstm_cell/forget/beta',
 'training_model/decoder/multi_rnn_cell/cell_0/layer_norm_basic_lstm_cell/forget/gamma',
 'training_model/decoder/multi_rnn_cell/cell_0/layer_norm_basic_lstm_cell/forget/beta',
 'training_model/decoder/multi_rnn_cell/cell_1/Attention_Wrapper/layer_norm_basic_lstm_cell/forget/gamma',
 'training_model/decoder/multi_rnn_cell/cell_1/Attention_Wrapper/layer_norm_basic_lstm_cell/forget/beta']

In [1264]:
df_all_dev.values[12]

array([ list(['and', 'it', 'be', 'the', 'one', 'going', 'for', 'one', 'stop', '<EOS>']),
       list(['and', 'the', 'bus', 'omg', '<EOS>']),
       list([8378, 13635, 14926, 7441, 6200, 9187, 434, 6200, 3425, 1]),
       list([8378, 7441, 11483, 11526, 1]), 353842], dtype=object)

In [1265]:
input_user = data_formatting.encodeSent(df_all_dev.values[1][0], vocab_dict)
#input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)

with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run(
        inf_train.final_outputs, 
        #inf_model_2.decoder_pred_decode,
        feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [ ]:
np.array([[data_formatting.decodeSent(q_0.predicted_ids.T[i][0], inv_map), 
          q_0.scores.T[i][0]]for i in range(inf_model_2_params['beam_width'])])

In [ ]:
np.array([[data_formatting.decodeSent(q_0.predicted_ids.T[i][0], inv_map), 
          q_0.scores.T[i][0]]for i in range(inf_model_2_params['beam_width'])])

In [ ]:
np.array([[data_formatting.decodeSent(q_0.predicted_ids.T[i][0], inv_map), 
          q_0.scores.T[i][0]]for i in range(inf_model_2_params['beam_width'])])

In [ ]:
np.array([[data_formatting.decodeSent(q_0.predicted_ids.T[i][0], inv_map), 
          q_0.scores.T[i][0]]for i in range(inf_model_2_params['beam_width'])])

In [ ]:
#data_formatting.decodeSent(q_0.beam_search_decoder_output.predicted_ids.T[0][0], inv_map)

In [21]:
batch_size = inf_model_2.inference_decoder._batch_size
beam_width = inf_model_2.inference_decoder._beam_width
end_token = inf_model_2.inference_decoder._end_token
length_penalty_weight = inf_model_2.inference_decoder._length_penalty_weight

In [22]:
name= None

In [23]:
time = 0

In [24]:
finished, first_inputs, initial_state = inf_model_2.inference_decoder.initialize()

In [25]:
first_inputs, initial_state

(<tf.Tensor 'training_model/embedding_lookup_1:0' shape=(?, 3, 512) dtype=float32>,
 BeamSearchDecoderState(cell_state=(LSTMStateTuple(c=<tf.Tensor 'training_model/Reshape:0' shape=(?, 3, 256) dtype=float32>, h=<tf.Tensor 'training_model/Reshape_1:0' shape=(?, 3, 256) dtype=float32>), AttentionWrapperState(cell_state=LSTMStateTuple(c=<tf.Tensor 'training_model/Reshape_2:0' shape=(?, 3, 256) dtype=float32>, h=<tf.Tensor 'training_model/Reshape_3:0' shape=(?, 3, 256) dtype=float32>), attention=<tf.Tensor 'training_model/Reshape_4:0' shape=(?, 3, 256) dtype=float32>, time=<tf.Tensor 'training_model/AttentionWrapperZeroState/zeros:0' shape=() dtype=int32>, alignments=<tf.Tensor 'training_model/Reshape_5:0' shape=(?, 3, ?) dtype=float32>, alignment_history=())), log_probs=<tf.Tensor 'zeros:0' shape=(?, 3) dtype=float32>, finished=<tf.Tensor 'training_model/zeros:0' shape=(?, 3) dtype=bool>, lengths=<tf.Tensor 'zeros_1:0' shape=(?, 3) dtype=int32>))

In [26]:
#first_inputs
#These are the word embeddings of the initial token (which for initialize should be from the encoder?)
#Shape is [batch_size, embedding_size]
#initial_state
#This is the initial state of the decoder, a tuple consisting of 
#1. the LSTM cell and hiddens states
#2. The attention states

In [27]:
#When we pass time !=0 to inference decoder, even with initial step results we get non-zero parent ids

#beam_search_output, beam_search_state, next_inputs, finished = \
#                                                inf_model_2.inference_decoder.step(time, first_inputs, initial_state)

In [28]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([first_inputs, initial_state, batch_size], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [29]:
q_0[0].shape

(1, 3, 512)

In [1266]:
#with ops.name_scope(name, "BeamSearchDecoderStep", (time, first_inputs, initial_state)):
    
cell_state = initial_state.cell_state

first_inputs = nest.map_structure(lambda inp: inf_model_2.inference_decoder._merge_batch_beams(inp, s=inp.shape[2:]), 
                                  first_inputs)

cell_state = nest.map_structure(inf_model_2.inference_decoder._maybe_merge_batch_beams, cell_state, 
                                inf_model_2.inference_decoder._cell.state_size)

In [1267]:
initial_state.cell_state

(LSTMStateTuple(c=<tf.Tensor 'training_model/Reshape:0' shape=(?, 3, 256) dtype=float32>, h=<tf.Tensor 'training_model/Reshape_1:0' shape=(?, 3, 256) dtype=float32>),
 AttentionWrapperState(cell_state=LSTMStateTuple(c=<tf.Tensor 'training_model/Reshape_2:0' shape=(?, 3, 256) dtype=float32>, h=<tf.Tensor 'training_model/Reshape_3:0' shape=(?, 3, 256) dtype=float32>), attention=<tf.Tensor 'training_model/Reshape_4:0' shape=(?, 3, 256) dtype=float32>, time=<tf.Tensor 'training_model/AttentionWrapperZeroState/zeros:0' shape=() dtype=int32>, alignments=<tf.Tensor 'training_model/Reshape_5:0' shape=(?, 3, ?) dtype=float32>, alignment_history=()))

In [1268]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([first_inputs, initial_state], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


ValueError: Fetch argument <tf.Tensor 'Reshape_44:0' shape=(?,) dtype=float32> cannot be interpreted as a Tensor. (Tensor Tensor("Reshape_44:0", shape=(?,), dtype=float32) is not an element of this graph.)

In [ ]:
q_0[0].shape

In [ ]:
##This goes through _cell first
#The output of cell_outputs is also present in the next_cell_state, the cell_outputs is the attention

#The cell_state includes, the cell and hidden states of the LSTM, the attention states, the alignment history, and timestep, 
#what does alignment history and attention states really mean? need to internalize...

cell_outputs, next_cell_state = inf_model_2.inference_decoder._cell(first_inputs, cell_state)

In [1269]:
cell_outputs

<tf.Tensor 'training_model/decoder/while/BeamSearchDecoderStep/output_projection/BiasAdd_1:0' shape=(?, 3, 17323) dtype=float32>

In [1270]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([cell_outputs, next_cell_state], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user, input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user), len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


ValueError: Fetch argument <tf.Tensor 'training_model/decoder/while/BeamSearchDecoderStep/output_projection/BiasAdd_1:0' shape=(?, 3, 17323) dtype=float32> cannot be interpreted as a Tensor. (Tensor Tensor("training_model/decoder/while/BeamSearchDecoderStep/output_projection/BiasAdd_1:0", shape=(?, 3, 17323), dtype=float32) is not an element of this graph.)

In [ ]:
q_0[0].shape

In [1271]:
#These functions convert the shape of the cell outputs and states from [batch_size, beam_width, embedding_size]
#to [batch_size*beam_width, embedding_size], essentially, reducing a dimensionality by flattening the nested structure
#the map structure fn is used because cell state is a complex tuple structure, 
#outputs is a simpe structure (should be able to just call flatten?)
cell_outputs = nest.map_structure(lambda out: inf_model_2.inference_decoder._split_batch_beams(out, out.shape[1:]), 
                                  cell_outputs)

next_cell_state = nest.map_structure(inf_model_2.inference_decoder._maybe_split_batch_beams, next_cell_state, 
                                     inf_model_2.inference_decoder._cell.state_size)

if inf_model_2.inference_decoder._output_layer is not None:
    cell_outputs = inf_model_2.inference_decoder._output_layer(cell_outputs)

ValueError: Unexpected behavior when reshaping between beam width and batch size.  The reshaped tensor has shape: (?, 3, 3, 256).  We expected it to have shape (batch_size, beam_width, depth) == (?, 3, 256).  Perhaps you forgot to create a zero_state with batch_size=encoder_batch_size * beam_width?

In [39]:
#cell_outputs_rnn_call, next_cell_state_rnn_call = inf_model_2.inference_decoder._cell.call(first_inputs, cell_state)

#cell_outputs_rnn_call_call, next_cell_state_rnn_call_call = inf_model_2.inference_decoder._cell.call(cell_outputs_rnn_call, 
#                                                                                                     next_cell_state_rnn_call)

In [40]:
sess.close()

In [41]:
static_batch_size = tensor_util.constant_value(batch_size)

prediction_lengths = initial_state.lengths
previously_finished = initial_state.finished

In [42]:
end_token = data_formatting.EOS

In [43]:
def _check_maybe(t):
  if isinstance(t, tensor_array_ops.TensorArray):
    raise TypeError(
        "TensorArray state is not supported by BeamSearchDecoder: %s" % t.name)
  if t.shape.ndims is None:
    raise ValueError(
        "Expected tensor (%s) to have known rank, but ndims == None." % t)

def _maybe_tensor_gather_helper(gather_indices, gather_from, batch_size,
                                range_size, gather_shape):
  """Maybe applies _tensor_gather_helper.
  This applies _tensor_gather_helper when the gather_from dims is at least as
  big as the length of gather_shape. This is used in conjunction with nest so
  that we don't apply _tensor_gather_helper to inapplicable values like scalars.
  Args:
    gather_indices: The tensor indices that we use to gather.
    gather_from: The tensor that we are gathering from.
    batch_size: The batch size.
    range_size: The number of values in each range. Likely equal to beam_width.
    gather_shape: What we should reshape gather_from to in order to preserve the
      correct values. An example is when gather_from is the attention from an
      AttentionWrapperState with shape [batch_size, beam_width, attention_size].
      There, we want to preserve the attention_size elements, so gather_shape is
      [batch_size * beam_width, -1]. Then, upon reshape, we still have the
      attention_size as desired.
  Returns:
    output: Gathered tensor of shape tf.shape(gather_from)[:1+len(gather_shape)]
      or the original tensor if its dimensions are too small.
  """
  _check_maybe(gather_from)
  if gather_from.shape.ndims >= len(gather_shape):
    return _tensor_gather_helper(
        gather_indices=gather_indices,
        gather_from=gather_from,
        batch_size=batch_size,
        range_size=range_size,
        gather_shape=gather_shape)
  else:
    return gather_from

def _tensor_gather_helper(gather_indices, gather_from, batch_size,
                          range_size, gather_shape):
  """Helper for gathering the right indices from the tensor.
  This works by reshaping gather_from to gather_shape (e.g. [-1]) and then
  gathering from that according to the gather_indices, which are offset by
  the right amounts in order to preserve the batch order.
  Args:
    gather_indices: The tensor indices that we use to gather.
    gather_from: The tensor that we are gathering from.
    batch_size: The input batch size.
    range_size: The number of values in each range. Likely equal to beam_width.
    gather_shape: What we should reshape gather_from to in order to preserve the
      correct values. An example is when gather_from is the attention from an
      AttentionWrapperState with shape [batch_size, beam_width, attention_size].
      There, we want to preserve the attention_size elements, so gather_shape is
      [batch_size * beam_width, -1]. Then, upon reshape, we still have the
      attention_size as desired.
  Returns:
    output: Gathered tensor of shape tf.shape(gather_from)[:1+len(gather_shape)]
  """
  range_ = array_ops.expand_dims(math_ops.range(batch_size) * range_size, 1)
  gather_indices = array_ops.reshape(gather_indices + range_, [-1])
  output = array_ops.gather(
      array_ops.reshape(gather_from, gather_shape), gather_indices)
  final_shape = array_ops.shape(gather_from)[:1 + len(gather_shape)]
  static_batch_size = tensor_util.constant_value(batch_size)
  final_static_shape = (tensor_shape.TensorShape([static_batch_size])
                        .concatenate(
                            gather_from.shape[1:1 + len(gather_shape)]))
  output = array_ops.reshape(output, final_shape)
  output.set_shape(final_static_shape)
  return output
#Have to understand how this works!
def _mask_probs(probs, eos_token, finished):
  """Masks log probabilities.
  The result is that finished beams allocate all probability mass to eos and
  unfinished beams remain unchanged.
  Args:
    probs: Log probabiltiies of shape `[batch_size, beam_width, vocab_size]`
    eos_token: An int32 id corresponding to the EOS token to allocate
      probability to.
    finished: A boolean tensor of shape `[batch_size, beam_width]` that
      specifies which
      elements in the beam are finished already.
  Returns:
    A tensor of shape `[batch_size, beam_width, vocab_size]`, where unfinished
    beams stay unchanged and finished beams are replaced with a tensor with all
    probability on the EOS token.
  """
  vocab_size = array_ops.shape(probs)[2]
  finished_mask = array_ops.expand_dims(
      math_ops.to_float(1. - math_ops.to_float(finished)), 2)
  # These examples are not finished and we leave them
  non_finished_examples = finished_mask * probs
  # All finished examples are replaced with a vector that has all
  # probability on EOS
  finished_row = array_ops.one_hot(
      eos_token,
      vocab_size,
      dtype=probs.dtype,
      on_value=0.,
      off_value=probs.dtype.min)
  finished_examples = (1. - finished_mask) * finished_row
  return finished_examples + non_finished_examples

In [44]:
logits = cell_outputs
step_log_probs = nn_ops.log_softmax(logits)

In [45]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([step_log_probs, logits, cell_outputs], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [46]:
q_0[0].shape, q_0[1].shape, q_0[2].shape

((1, 3, 17323), (1, 3, 17323), (1, 3, 17323))

In [47]:
step_log_probs

<tf.Tensor 'Reshape_15:0' shape=(?, 3, 17323) dtype=float32>

In [48]:
tf.shape(step_log_probs)[0]

<tf.Tensor 'strided_slice_14:0' shape=() dtype=int32>

In [49]:
step_log_probs = _mask_probs(step_log_probs, end_token, previously_finished)

In [50]:
total_probs = array_ops.expand_dims(initial_state.log_probs, 2) + step_log_probs

In [51]:
#total_probs_1 = total_probs-1

In [52]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([total_probs, previously_finished], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [53]:
np.argsort(q_0[0][0][0])

array([ 3802, 11741, 13899, ..., 11678, 10359,  4400], dtype=int64)

In [54]:
q_0[0][0][0][4400]

-0.8556143

In [55]:
vocab_size = logits.shape[-1].value or array_ops.shape(logits)[-1]

lengths_to_add = array_ops.one_hot(
      indices=array_ops.tile(
          array_ops.reshape(end_token, [1, 1]), [batch_size, beam_width]),
      depth=vocab_size,
      on_value=0,
      off_value=1)

In [56]:
sess.close()

In [57]:
def _get_scores(log_probs, sequence_lengths, length_penalty_weight, time):
    """Calculates scores for beam search hypotheses.
    Args:
    log_probs: The log probabilities with shape
    `[batch_size, beam_width, vocab_size]`.
    sequence_lengths: The array of sequence lengths.
    length_penalty_weight: Float weight to penalize length. Disabled with 0.0.
    Returns:
    The scores normalized by the length_penalty.
    """
    length_penality_ = _length_penalty(sequence_lengths=sequence_lengths, penalty_factor=length_penalty_weight)

    score = log_probs/length_penality_

    current_time = tf.contrib.util.constant_value(time)

    if current_time==0:
        
        sorted_tokens = sorted([i[0] for i in n_grams[0] if i[0] in inv_map.keys()])

        X = tf.constant(sorted_tokens)
        y, idx, cnts = tf.unique_with_counts(X)
        cnts_prob = tf.log(cnts/len(sorted_tokens))

        prob_T = {a[i]:b[i] for i in range(len(a))}

        p_T = [-1000  if i not in a else prob_T[i] for i in range(len(inv_map))]
        score = score + p_T
    return score


In [58]:
def _length_penalty(sequence_lengths, penalty_factor):
  """Calculates the length penalty. See https://arxiv.org/abs/1609.08144.
  Args:
    sequence_lengths: The sequence length of all hypotheses, a tensor
      of shape [beam_size, vocab_size].
    penalty_factor: A scalar that weights the length penalty.
  Returns:
    The length penalty factor, a tensor fo shape [beam_size].
  """
  penalty_factor = ops.convert_to_tensor(penalty_factor, name="penalty_factor")
  penalty_factor.set_shape(())  # penalty should be a scalar.
  static_penalty = tensor_util.constant_value(penalty_factor)
  if static_penalty is not None and static_penalty == 0:
    return 1.0
  return math_ops.div((5. + math_ops.to_float(sequence_lengths))
                      **penalty_factor, (5. + 1.)**penalty_factor)

In [59]:
add_mask = (1 - math_ops.to_int32(previously_finished))

lengths_to_add = array_ops.expand_dims(add_mask, 2) * lengths_to_add

new_prediction_lengths = (lengths_to_add + array_ops.expand_dims(prediction_lengths, 2))

In [60]:
n_grams = pickle.load(open('n_grams_test.pkl', 'rb'))

In [61]:
sorted_tokens = sorted([i[0] for i in n_grams[0] if i[0] in inv_map.keys()])

In [62]:
a, b= np.unique(sorted_tokens, return_counts=True)

In [63]:

X = tf.constant(sorted_tokens)
y, idx, cnts = tf.unique_with_counts(X)
cnts_prob = cnts/len(sorted_tokens)
with tf.Session() as sess:
    a, _, b = sess.run([y, idx, cnts_prob])
    print (a)
    print (b)

[    3     5     6 ..., 17320 17321 17322]
[  1.10949002e-05   5.54745011e-06   5.54745011e-05 ...,   1.66423503e-05
   5.54745011e-06   1.10949002e-05]


In [64]:
prob_T = {a[i]:np.log(b[i]/len(sorted_tokens)) for i in range(len(a))}

p_T = [-100  if i not in a else prob_T[i]*0.1 for i in range(len(inv_map))]

In [65]:
p_T[4400]*.1

-0.19000337662121669

In [66]:
#scores = total_probs + p_T
scores = total_probs
scores_shape = array_ops.shape(scores)

time = ops.convert_to_tensor(time, name="time")

scores_flat = control_flow_ops.cond(time > 0,
      lambda: array_ops.reshape(scores, [batch_size, -1]), lambda: scores[:, 0])

num_available_beam = control_flow_ops.cond(
      time > 0, lambda: math_ops.reduce_prod(scores_shape[1:]),
      lambda: math_ops.reduce_prod(scores_shape[2:]))

In [67]:
scores_flat = scores_flat + p_T

In [68]:
# Pick the next beams according to the specified successors function
next_beam_size = math_ops.minimum(
  ops.convert_to_tensor(beam_width, dtype=dtypes.int32, name="beam_width"),
  num_available_beam)

next_beam_scores, word_indices = nn_ops.top_k(scores_flat, k=next_beam_size)
next_beam_scores.set_shape([static_batch_size, beam_width])
word_indices.set_shape([static_batch_size, beam_width])

In [69]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([scores_flat], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [70]:
q_0[0][0][4400], p_T[4400] 

(-2.7556474, -1.9000337662121667)

In [71]:
q_0[0][0][4400], p_T[4400] + -0.85561597

(-2.7556474, -2.7556497362121668)

In [72]:
q_0[0][0][4400], p_T[4400] + -0.85561597

(-2.7556474, -2.7556497362121668)

In [73]:
np.argsort(q_0[0])

array([[    0, 10150,  2557, ..., 17149, 10359,  4400]], dtype=int64)

In [74]:
q_0

[array([[-133.68927002, -127.15158081, -127.81147003, ...,  -19.06400108,
          -28.31323433,  -30.63676834]], dtype=float32)]

In [75]:
next_beam_probs = _tensor_gather_helper(
      gather_indices=word_indices,
      gather_from=scores,
      batch_size=batch_size,
      range_size=beam_width * vocab_size,
      gather_shape=[-1])

#Here, word indices represents the positions of the flattened list of beams, which goes from 1 to beam_width*vocab_size
#thus the word indices obtained are not the word indices of the 1 to N vocab but the flattened out list of logprobs from 1 to 
#beam_width*vocab_size
next_word_ids = math_ops.to_int32(word_indices % vocab_size)
#The beam_ids represent which beams these word indices (and thereby log prob values belong to), as
#we have word indices extracted out of 1 to beam_width*vocab_size
next_beam_ids = math_ops.to_int32(word_indices / vocab_size)

In [76]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([next_word_ids, next_beam_ids, scores, scores_flat, next_beam_scores, next_beam_probs,
                    word_indices, next_beam_size], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [77]:
[inv_map[i] for i in q_0[0][0]]

['last', 'hey', 'my']

In [78]:
[inv_map[i] for i in [17149,  4400, 10359]]

['my', 'last', 'hey']

In [79]:
np.argmax(q_0[2][0][0])

4400

In [80]:
q_0[3][0][np.argsort(q_0[3][0])[-3:]]

array([-4.42817593, -4.01685619, -2.75564575], dtype=float32)

In [81]:
-9.57212543 + -2.60707283

-12.17919826

In [82]:
-7.75377798 + -2.0549202

-9.80869818

In [83]:
-7.49675941 + -0.85561192

-8.35237133

In [84]:
q_0[3][0][np.argsort(q_0[3][0])[-3:]]

array([-4.42817593, -4.01685619, -2.75564575], dtype=float32)

In [85]:
q_0[3][0][:vocab_size], q_0[3][0][vocab_size:vocab_size*2], q_0[3][0][vocab_size*2:]

(array([-133.68927002, -127.15158844, -127.81147003, ...,  -19.06400299,
         -28.31324005,  -30.63677025], dtype=float32),
 array([], dtype=float32),
 array([], dtype=float32))

In [86]:
q_0[4], q_0[5]

(array([[-2.75564575, -4.01685619, -4.42817593]], dtype=float32),
 array([[-0.85561204, -2.05491853, -2.74253583]], dtype=float32))

In [87]:
q_0[0], q_0[1]

(array([[ 4400, 10359, 17149]]), array([[0, 0, 0]]))

In [88]:
previously_finished = _tensor_gather_helper(
      gather_indices=next_beam_ids,
      gather_from=previously_finished,
      batch_size=batch_size,
      range_size=beam_width,
      gather_shape=[-1])

next_finished = math_ops.logical_or(previously_finished, math_ops.equal(next_word_ids, end_token))

In [89]:
lengths_to_add = math_ops.to_int32(math_ops.not_equal(next_word_ids, end_token))
lengths_to_add = (1 - math_ops.to_int32(next_finished)) * lengths_to_add

In [90]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([previously_finished, next_finished, lengths_to_add], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [91]:
q_0

[array([[False, False, False]], dtype=bool),
 array([[False, False, False]], dtype=bool),
 array([[1, 1, 1]])]

In [92]:
next_prediction_len = _tensor_gather_helper(
      gather_indices=next_beam_ids,
      gather_from=initial_state.lengths,
      batch_size=batch_size,
      range_size=beam_width,
      gather_shape=[-1])

next_prediction_len += lengths_to_add

In [93]:
with tf.Session() as sess:
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run([next_prediction_len],
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})    

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [94]:
q_1

[array([[1, 1, 1]])]

In [95]:
next_cell_state = nest.map_structure(
      lambda gather_from: _maybe_tensor_gather_helper(
          gather_indices=next_beam_ids,
          gather_from=gather_from,
          batch_size=batch_size,
          range_size=beam_width,
          gather_shape=[batch_size * beam_width, -1]),
      next_cell_state)

In [96]:
with tf.Session() as sess:
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run([next_cell_state],
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})    

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [97]:
#These are the returns, next_state and output to the step function, but how exactly do we transpose this back and push it
#into cell? Moreover what is teh mechanics that determines when the beams become different? I.E. are no longer identical

In [98]:
beam_search_state = tf.contrib.seq2seq.BeamSearchDecoderState(
      cell_state=next_cell_state,
      log_probs=next_beam_probs,
      lengths=next_prediction_len,
      finished=next_finished)

beam_search_output = tf.contrib.seq2seq.BeamSearchDecoderOutput(
      scores=next_beam_scores,
      predicted_ids=next_word_ids,
      parent_ids=next_beam_ids)

In [100]:
with tf.Session() as sess:
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run([beam_search_state],
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})    

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [101]:
#Here we define the returns
finished = beam_search_state.finished
next_finished = finished
sample_ids = beam_search_output.predicted_ids

In [102]:
#This part is most important, look up the new seq terms and take their embedding
#We pass that onto the decoder cell network in the next loop iteration

next_inputs = tf.cond(tf.reduce_all(finished), lambda: inf_model_2.inference_decoder._start_inputs,
              lambda: inf_model_2.inference_decoder._embedding_fn(sample_ids))

In [103]:
beam_search_output.predicted_ids

<tf.Tensor 'mod:0' shape=(?, 3) dtype=int32>

In [104]:
def _shape(batch_size, from_shape):
    if not isinstance(from_shape, tf.TensorShape):
        return tensor_shape.TensorShape(None)
    else:
        batch_size = tf.contrib.util.constant_value(tf.convert_to_tensor(batch_size, name="batch_size"))
        return tf.TensorShape([batch_size]).concatenate(from_shape)

In [105]:
def _create_ta(s, d):
    return tf.TensorArray(
        dtype=d, size=0, dynamic_size=True, element_shape=_shape(batch_size, s))

In [106]:
initial_outputs_ta = nest.map_structure(_create_ta, inf_model_2.inference_decoder.output_size,
                                                    inf_model_2.inference_decoder.output_dtype)            

In [107]:
#return time, outputs, state, inputs, finished, sequence_lengths

initial_outputs_ta = nest.map_structure(lambda ta, out: ta.write(time, out),
                          initial_outputs_ta, beam_search_output)

In [114]:
final_outputs = nest.map_structure(lambda ta: ta.stack(), initial_outputs_ta)

In [122]:
with tf.Session() as sess:
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run(final_outputs.predicted_ids,
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})    

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [123]:
#final_outputs.predicted_ids , 
#array([[ 4400, 10359, 17149]])

array([[ 4400, 10359, 17149]])

In [108]:
#next_sequence_lengths = tf.where(
#  tf.logical_and(tf.logical_not(finished), next_finished),
#      tf.fill(tf.shape(sequence_lengths), time + 1),
#  sequence_lengths)

In [ ]:
finished = beam_search_state.finished

In [ ]:
sample_ids = beam_search_output.predicted_ids

In [252]:
sample_ids

NameError: name 'sample_ids' is not defined

In [115]:
next_inputs = control_flow_ops.cond(math_ops.reduce_all(finished), 
                                    lambda: inf_model_2.inference_decoder._start_inputs,
                                    lambda: inf_model_2.inference_decoder._embedding_fn(sample_ids))

In [122]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    #sess.run(tf.global_variables_initializer())

    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run([next_inputs, beam_search_state, sample_ids],
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]}) 

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [123]:
q_1[2]

array([[ 4400, 10359, 11678]])

In [118]:
q_1[0].shape

(1, 3, 512)

In [119]:
q_0[1][3].shape, q_1[1][3].shape

IndexError: index 3 is out of bounds for axis 0 with size 1

In [ ]:
q_0[1][2].shape, q_1[1][2].shape

In [ ]:
q_0[1][1].shape, q_1[1][1].shape

In [ ]:
len(q_0[1][0]), len(q_1[1][0])

In [ ]:
#next_inputs
#These are the chosen word ids which we have performed the embedding look up for
first_inputs = next_inputs[0]#The [0] is for batch size, should not need it

#next_states
#these are the next states of the decoder network which pass back through the process
initial_state = next_state

finished = finished

In [217]:
time =1

In [124]:
import tensorflow as tf
import pickle

In [404]:
#outputs.predicted_id at first pass time=1
#array([[ 4400, 10359, 17149]])
#time =1
time = 2
tf.reset_default_graph()

In [824]:
n_grams = pickle.load(open('n_grams_test.pkl', 'rb'))

n_grams_tf = tf.constant(n_grams[time])

beam_length = n_grams_tf.shape[1].value

#test = tf.constant([4400, 10359, 17149])
beam = tf.constant([[6178, 5729], [8847, 16625], [12377,  6605]])

beam_pad = tf.pad(beam, [[0, 0], [0, 1]], mode='CONSTANT', constant_values=0)

n_grams_tf.shape[0].value

multiply = tf.constant([1, n_grams_tf.shape[0].value])

beam_pad_matrix = tf.tile(beam_pad, multiply)

beam_pad_matrix_reshape = tf.reshape(beam_pad_matrix, \
                                     [beam_length, n_grams_tf.shape[0].value, beam_length])

multiply = tf.constant([beam.shape[0].value, 1])

matrix = tf.tile(n_grams_tf, multiply)

matrix_reshape = tf.reshape(matrix, [beam_length, n_grams_tf.shape[0].value, beam_length])

y_matrix_reshape = tf.subtract(matrix_reshape, beam_pad_matrix_reshape)

y_indices = tf.to_int32(tf.equal(tf.zeros([1, beam_length], dtype=tf.int32), y_matrix_reshape))

y_indices_reduce_sum = tf.reduce_sum(y_indices, axis=2)

y_args = tf.where(tf.equal(2, y_indices_reduce_sum))

mask_test = tf.boolean_mask(matrix_reshape, tf.equal(tf.zeros([1, beam_length], dtype=tf.int32), y_matrix_reshape))

vocab_count_template = tf.Variable(tf.zeros([3, vocab_size], dtype=tf.int32))

In [1094]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    q = sess.run([vocab_count_template, wwww, y_args, y_indices_reduce_sum, y_indices, y_matrix_reshape ,matrix_reshape, beam_pad_matrix_reshape])

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import pickle
time = 2
n_grams = pickle.load(open('n_grams_test.pkl', 'rb'))

n_grams_tf = tf.constant(n_grams[time])

In [3]:
beam_length = time + 1

#test = tf.constant([4400, 10359, 17149])
beam = tf.constant([[6178, 5729], [8847, 16625], [12377,  6605]])

beam_pad = tf.pad(beam, [[0, 0], [0, 1]], mode='CONSTANT', constant_values=0)

In [4]:
current_beam = tf.constant(0)
n_beams = tf.constant(3)

In [5]:
yt = tf.subtract(n_grams_tf, beam_pad[0])

In [44]:
#test  = tf.Variable(np.reshape(np.array([0 for i in range(180261*3)]), (180261, 3)), dtype=tf.int32, name='test')

In [377]:
test = tf.Variable([0 for i in range(17322)], dtype=tf.float32)

In [10]:
initial_beam_step = tf.constant(0)

In [452]:
qq = tf.contrib.util.constant_value(current_beam)

In [104]:
#When you specify the axis of concat such a rank must exist! 
#Thus cannot specify axis=1 if the rank is 0!
scatter_base = tf.constant([17322])
concat_base = tf.constant([[1 for i in range(17322)]])

In [138]:
def body(test, n_grams_tf, beam_pad, current_beam, n_beams):
    #This loops on each beam individually
    
    #test = tf.concat([test, beam[time]], axis=0)
    
    #Find where current beam matches n_gram sequence up to current seq pos, cast as int
    y_test = tf.to_int32(tf.equal(n_grams_tf, beam_pad[current_beam]))
    #Reduce across the length of the beam 
    y_test_reduce_sum = tf.reduce_sum(y_test, axis=1)
    #Find args where the beam is matched to the n_gram combinations
    y_args = tf.where(tf.equal(2, y_test_reduce_sum))
    #Grab the n_gram sequences that are matched
    y_diff_gather = tf.gather(n_grams_tf, y_args)
    #Take the last token in each sequence and count their unique occurrences
    y_last_token = y_diff_gather[:,0][:,time]
    y_last_token_unique = tf.unique_with_counts(y_last_token)

    stack_index = tf.fill([tf.shape(y_last_token_unique.y)[0]], current_beam)

    #concat_y_last_token_unique_count = tf.stack([stack_index, y_last_token_unique.y], axis=1)

    test_add = tf.scatter_nd(indices=y_last_token_unique.y, updates=y_last_token_unique.count, shape=scatter_base)
    
    test = tf.concat([test, [test_add]], axis=0)
    
    #Initialize and create a zero element vector of length vocabulary and add the obtained frequency counts
    #to the indexes which represent token ids, normalize to get probability
   # vocab = tf.Variable(tf.zeros([len(vocab_dict)], dtype=tf.int32))
    #with tf.variable_scope('vocab', reuse=None):
   # qq = tf.contrib.util.constant_value(current_beam)
  # # vocab = tf.get_variable(name='vocab5_%d' % qq , 
   #                         shape=(len(vocab_dict)), dtype=tf.int32,
   #                         initializer=tf.zeros_initializer())

   # vocab_with_probs = tf.scatter_add(vocab, y_last_token_unique.y,
      #                                    y_last_token_unique.count)/tf.reduce_sum(y_last_token_unique.count)
    
   # test = tf.to_float(vocab_with_probs)
    #test = tf.concat([test, tf.to_float(vocab_with_probs)], axis=0)
    
    return test, n_grams_tf, beam_pad, current_beam+1, n_beams

def condition(test, n_grams_tf, beam_pad, current_beam, n_beams):
    return current_beam < 3

In [139]:
outputs = tf.while_loop(condition, body, 
                        [concat_base, n_grams_tf, beam_pad, initial_beam_step, n_beams], 
                        shape_invariants=[tf.TensorShape([None, 17322]), n_grams_tf.get_shape(),
                                      beam_pad.get_shape(), initial_beam_step.get_shape(), n_beams.get_shape()])

In [140]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    #q = sess.run(outputs[0].stack())
    q = sess.run(outputs)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [141]:
np.argsort(q[0][1])

array([    0, 11537, 11538, ...,  5769,  5775, 15173], dtype=int64)

In [438]:
np.argsort(q[0]), q[0][3380]

(array([    0, 11494, 11495, ..., 14250, 15173,  3380], dtype=int64),
 0.086065575)

In [356]:
q[0][0]

0.0

In [282]:
q[0][0][3380]

0.25819671

In [161]:
time = tf.constant(2)

In [464]:
test_3 = tf.Variable([[0 for i in range(17322)], [0 for i in range(17322)] ,[0 for i in range(17322)]], dtype=tf.int32)

In [113]:
#Find where current beam matches n_gram sequence up to current seq pos, cast as int
y_test = tf.to_int32(tf.equal(n_grams_tf, beam_pad[0]))
#Reduce across the length of the beam 
y_test_reduce_sum = tf.reduce_sum(y_test, axis=1)
#Find args where the beam is matched to the n_gram combinations
y_args = tf.where(tf.equal(2, y_test_reduce_sum))
#Grab the n_gram sequences that are matched
y_diff_gather = tf.gather(n_grams_tf, y_args)
#Take the last token in each sequence and count their unique occurrences
y_last_token = y_diff_gather[:,0][:,time]
y_last_token_unique = tf.unique_with_counts(y_last_token)

In [ ]:
#Initialize and create a zero element vector of length vocabulary and add the obtained frequency counts
#to the indexes which represent token ids, normalize to get probability
#vocab = tf.Variable(tf.zeros([len(vocab_dict)], dtype=tf.int32))
#vocab = tf.Variable(tf.zeros([len(vocab_dict)], dtype=tf.int32))
#vocab = tf.get_variable(name='vocab_b', shape=(len(vocab_dict)), dtype=tf.int32,
#                      initializer=tf.zeros_initializer())

In [467]:
#vocab_with_log_probs = tf.scatter_add(test_3, y_last_token_unique.y,
 #                                     y_last_token_unique.count)/tf.reduce_sum(y_last_token_unique.count)

In [109]:
#First, specify the dimensions of the scattered tensor, here this means its 3x4 ie
# [0, 0, 0, 0
#  0, 0, 0, 0
#  0, 0, 0, 0]
test_3 = tf.constant([17322])
#Next indices specifies the coordinates
#So [[0, 2], [1, 2]] specifies two indices
#at zeroth row, 2nd column (counting from zero)
#and first row, 2nd column
indices = tf.constant([[    0, 15173],
       [    0, 15259],
       [    0, 11599]])
#Next, specify the actual values, to be inserted into these positions
updates = tf.constant([1, 2, 3])

In [110]:
stack_base = tf.constant([1 for i in range(17322)])

In [111]:
#stack_index = tf.fill([tf.shape(y_last_token_unique.y)[0]], 2)

#concat_y_last_token_unique_count = tf.stack([stack_index, y_last_token_unique.y], axis=1)

In [114]:
test_add= tf.scatter_nd(indices=y_last_token_unique.y, updates=y_last_token_unique.count, shape=test_3)

In [115]:
concat_base = tf.constant([[1 for i in range(17322)]])

In [121]:
test_add_nest = [test_add]

In [124]:
tf_concat = tf.concat([concat_base, [test_add]], axis=0)

In [125]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    w = sess.run([tf_concat])

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [128]:
np.max(w[0][1])

48

In [661]:
np.argsort(w[0])

array([    0, 11537, 11538, ...,  5769,  5775, 15173], dtype=int64)

In [552]:
#First subtract the current (padded) beam from the list of n_gram combinations
#y_diff = tf.subtract(n_grams_tf, beam_pad[0])
#
#y_segments = y_diff[:,0]
#y_data = y_diff[:,1]
#
#y_indices = tf.to_int32(tf.equal(tf.zeros([1, beam_length], dtype=tf.int32), y_diff))
#y_indices_reduce_sum = tf.reduce_sum(y_indices, axis=1)
#y_args = tf.where(tf.equal(2, y_indices_reduce_sum))

#y_diff_gather = tf.gather(n_grams_tf, y_args)

#y_last_token = y_diff_gather[:,0][:,time]

#y_last_token_unique = tf.unique_with_counts(y_last_token)

In [142]:
vocab = tf.Variable(tf.zeros([len(vocab_dict)], dtype=tf.int32))

vocab_with_log_probs = tf.scatter_add(vocab, y_last_token_unique.y, y_last_token_unique.count)

In [149]:
#y_segments_unique = tf.unique_with_counts(y_segments)

#y_segments_unique_args = tf.where(tf.equal(2, y_segments_unique.count))

#y_2_slice_y = tf.gather(n_grams_tf, tf.gather(y_segments_unique.y, y_segments_unique_args))

In [156]:
q[0]

array([[1, 1, 0],
       [0, 0, 0],
       [0, 0, 0],
       ..., 
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [148]:
q[3][3380]

63

In [99]:
q[1].shape, q[0].shape, q[2].shape

((180261, 3), (732, 1), (180261,))

In [64]:
multiply = tf.constant([1, n_grams_tf.shape[0].value])

beam_pad_matrix = tf.tile(beam_pad, multiply)

beam_pad_matrix_reshape = tf.reshape(beam_pad_matrix, \
                                     [beam_length, n_grams_tf.shape[0].value, beam_length])

multiply = tf.constant([beam.shape[0].value, 1])

matrix = tf.tile(n_grams_tf, multiply)

In [81]:
matrix_reshape = tf.reshape(matrix, [beam_length, n_grams_tf.shape[0].value, beam_length])

y_matrix_reshape = tf.subtract(matrix_reshape, beam_pad_matrix_reshape)

y_indices = tf.to_int32(tf.equal(tf.zeros([1, beam_length], dtype=tf.int32), y_matrix_reshape))

#y_indices_reduce_sum = tf.reduce_sum(y_indices, axis=2)

In [66]:
y_indices_reduce_sum.get_shape()

TensorShape([Dimension(3), Dimension(180261)])

In [63]:

mask_test = tf.boolean_mask(matrix_reshape, tf.equal(tf.zeros([1, beam_length], dtype=tf.int32), y_matrix_reshape))

vocab_count_template = tf.Variable(tf.zeros([3, vocab_size], dtype=tf.int32))

NameError: name 'vocab_size' is not defined

In [64]:
q

[1, array([[ 1.,  1.],
        [ 1.,  1.],
        [ 1.,  1.],
        [ 1.,  1.]], dtype=float32)]

In [1066]:
wwww = tf.as_string(www)

In [1039]:
xxx = tf.unique_with_counts(www)

In [1032]:
www =tf.gather_nd(matrix_reshape, y_args)

In [1085]:
z = tf.map_fn(lambda x: x, wwww)

In [1053]:
q[0]

array([[ 6178,  5729, 15173],
       [ 6178,  5729, 15259],
       [ 6178,  5729, 11599],
       ..., 
       [ 8847, 16625, 15228],
       [ 8847, 16625, 12377],
       [12377,  6605, 14362]])

In [1044]:
q[2]

array([[2, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 2, 0, 0],
       [0, 0, 0, ..., 0, 0, 2]])

In [1048]:
q[5]

array([[[ 6178,  5729, 15173],
        [ 5729, 15173,  4193],
        [15173,  4193,  1001],
        ..., 
        [ 8847, 16625, 12377],
        [16625, 12377,  6605],
        [12377,  6605, 14362]],

       [[ 6178,  5729, 15173],
        [ 5729, 15173,  4193],
        [15173,  4193,  1001],
        ..., 
        [ 8847, 16625, 12377],
        [16625, 12377,  6605],
        [12377,  6605, 14362]],

       [[ 6178,  5729, 15173],
        [ 5729, 15173,  4193],
        [15173,  4193,  1001],
        ..., 
        [ 8847, 16625, 12377],
        [16625, 12377,  6605],
        [12377,  6605, 14362]]])

In [669]:
#zero_pad = tf.Variable(tf.zeros([time+1], dtype=tf.int32))
#beam_zero_pad = tf.scatter_add(zero_pad, [i for i in range(time)], test[1])

In [448]:
y = tf.where(tf.equal([0], n_grams_tf - vocab_with_probs))

In [469]:
y_segments = y[:,0]
y_data = y[:,1]

In [567]:
y_segments_unique = tf.unique_with_counts(y_segments)

y_segments_unique_args = tf.where(tf.equal(2, y_segments_unique.count))

y_2_slice_y = tf.gather(n_grams_tf, tf.gather(y_segments_unique.y, y_segments_unique_args))

In [621]:
y_last_seq_unique = tf.unique_with_counts(y_2_slice_y[:,0][:,2])

In [624]:
y_last_seq_prob = y_last_seq_unique.count/tf.shape(y_2_slice_y)[0]

In [643]:
vocab = tf.Variable(tf.zeros([vocab_size], dtype=tf.int32))

vocab_with_log_probs = tf.log(tf.scatter_add(vocab, y_last_seq_unique.y, y_last_seq_unique.count)/tf.shape(y_2_slice_y)[0])

In [644]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    q = sess.run([y, y_segments_unique, y_segments_unique_args, y_2_slice_y, y_last_seq_unique, vocab_with_log_probs])

In [642]:
q[5][3380]

-2.4526457875697787

In [ ]:
###############################

In [452]:
y_2_slice_y = tf.gather(y, y_2[:,0])[:,0]

In [453]:
y_2_slice_y_slice_n_grams_tf = tf.gather(n_grams_tf, y_2_slice_y)

In [454]:
y_2_slice_y_slice_n_grams_tf_unique = tf.unique_with_counts(y_2_slice_y_slice_n_grams_tf[:,1])

In [458]:
#y_norm = y_2_slice_y_slice_n_grams_tf_unique.count/count

In [459]:
vocab = tf.Variable(tf.zeros([vocab_size], dtype=tf.int32))

vocab_with_probs = tf.scatter_add(vocab, y_2_slice_y_slice_n_grams_tf_unique.y, y_2_slice_y_slice_n_grams_tf_unique.count)/

In [464]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    t = sess.run([vocab_with_probs, vocab])
    #q = sess.run(yy_string)
    q = sess.run([y_2_slice_y_slice_n_grams_tf, y_2, y, n_grams_tf, test[1]])

In [463]:
q

[array([[ 6178,  5729, 15173],
        [ 6178,  5729, 15259],
        [ 6178,  5729, 11599],
        ..., 
        [ 6178,  5729, 17199],
        [ 7338,  5729,  6957],
        [ 6178,  5729, 17199]]), array([[   1,    1],
        [  22,    1],
        [  24,    1],
        ..., 
        [5602,    1],
        [5604,    1],
        [5623,    1]], dtype=int64), array([[     0,      0],
        [     0,      1],
        [     9,      0],
        ..., 
        [180183,      0],
        [180183,      1],
        [180233,      0]], dtype=int64), array([[ 6178,  5729, 15173],
        [ 5729, 15173,  4193],
        [15173,  4193,  1001],
        ..., 
        [ 8847, 16625, 12377],
        [16625, 12377,  6605],
        [12377,  6605, 14362]]), array([6178, 5729])]

In [401]:
q[1]

array([[ 6178,  5729, 15173],
       [ 5729, 15173,  4193],
       [15173,  4193,  1001],
       ..., 
       [ 8847, 16625, 12377],
       [16625, 12377,  6605],
       [12377,  6605, 14362]])